In [ ]:
import argparse
import base64
from datetime import datetime
import os
import shutil
import numpy as np
import socketio
import eventlet
import eventlet.wsgi
from PIL import Image
from flask import Flask
from io import BytesIO
from PIL import Image as im

from keras.models import load_model
import h5py
from keras import __version__ as keras_version

sio = socketio.Server()
app = Flask(__name__)
model = None
prev_image_array = None
image_folder='saveimage'
modelfile="model.h5"

class SimplePIController:
    def __init__(self, Kp, Ki):
        self.Kp = Kp
        self.Ki = Ki
        self.set_point = 0.
        self.error = 0.
        self.integral = 0.

    def set_desired(self, desired):
        self.set_point = desired

    def update(self, measurement):
        # proportional error
        self.error = self.set_point - measurement

        # integral error
        self.integral += self.error

        return self.Kp * self.error + self.Ki * self.integral


controller = SimplePIController(0.1, 0.002)
set_speed = 9
controller.set_desired(set_speed)


@sio.on('telemetry')
def telemetry(sid, data):
    if data:
        # The current steering angle of the car
        steering_angle = data["steering_angle"]
        # The current throttle of the car
        throttle = data["throttle"]
        # The current speed of the car
        speed = data["speed"]
        # The current image from the center camera of the car
        imgString = data["image"]
        image = Image.open(BytesIO(base64.b64decode(imgString)))
        image_array = np.asarray(image)
        steering_angle = float(model.predict(image_array[None, :, :, :], batch_size=1))
        throttle = controller.update(float(speed))

        print(steering_angle, throttle)
        send_control(steering_angle, throttle)

        # save frame
        if image_folder != '':
            timestamp = datetime.utcnow().strftime('%Y_%m_%d_%H_%M_%S_%f')[:-3]
            image_filename = os.path.join(image_folder, timestamp)
            image.save('{}.jpg'.format(image_filename))
    else:
        # NOTE: DON'T EDIT THIS.
        sio.emit('manual', data={}, skip_sid=True)


@sio.on('connect')
def connect(sid, environ):
    print("connect ", sid)
    send_control(0, 0)


def send_control(steering_angle, throttle):
    sio.emit(
        "steer",
        data={
            'steering_angle': steering_angle.__str__(),
            'throttle': throttle.__str__()
        },
        skip_sid=True)


if __name__ == '__main__':
    # check that model Keras version is same as local Keras version
    f = h5py.File("model.h5", mode='r')
    model_version = f.attrs.get('keras_version')
    keras_version = str(keras_version).encode('utf8')

    if model_version != keras_version:
        print('You are using Keras version ', keras_version,
              ', but the model was built using ', model_version)

    model = load_model(modelfile)

    if image_folder != '':
        print("Creating image folder at {}".format(image_folder))
        if not os.path.exists(image_folder):
            os.makedirs(image_folder)
        else:
            shutil.rmtree(image_folder)
            os.makedirs(image_folder)
        print("RECORDING THIS RUN ...")
    else:
        print("NOT RECORDING THIS RUN ...")

    # wrap Flask application with engineio's middleware
    app = socketio.Middleware(sio, app)

    # deploy as an eventlet WSGI server
    eventlet.wsgi.server(eventlet.listen(('', 4567)), app)


Using TensorFlow backend.


Creating image folder at saveimage
RECORDING THIS RUN ...


(8540) wsgi starting up on http://0.0.0.0:4567
(8540) accepted ('127.0.0.1', 65148)


connect  008e1208f94d40969bb416c8f26fb8a4
-0.16005489230155945 0.918
-0.16005489230155945 0.936
-0.16005489230155945 0.9540000000000001
-0.10569101572036743 0.52524
-0.10569101572036743 0.5344800000000001
-0.10569101572036743 0.54372
-0.21994081139564514 0.15088620000000008
-0.21994081139564514 0.15224240000000006
-0.21994081139564514 0.15359860000000009
-0.20507876574993134 0.7576422
-0.20507876574993134 0.7708158
-0.20507876574993134 0.7839894000000001
-0.20245274901390076 0.9252648
-0.20245274901390076 0.9235592
-0.20245274901390076 0.9389036
-0.19996263086795807 0.8692514000000001
-0.19996263086795807 0.8829292000000001
-0.19996263086795807 0.896607
-0.20691390335559845 0.7856000000000001
-0.20691390335559845 0.7968330000000001
-0.20691390335559845 0.8080660000000001
-0.21879079937934875 0.6564764000000001
-0.21879079937934875 0.6645168
-0.21879079937934875 0.6725572
-0.21426160633563995 0.4941518
-0.21426160633563995 0.49853639999999994
-0.21426160633563995 0.502921
-0.13880632817

-0.0317327044904232 0.07541259999999991
-0.0317327044904232 0.07532519999999991
-0.0317327044904232 0.07523779999999991
-0.04110083729028702 0.07542579999999983
-0.04110083729028702 0.07534379999999984
-0.04110083729028702 0.07526179999999984
-0.06653907895088196 0.07555719999999987
-0.06653907895088196 0.07548259999999986
-0.06653907895088196 0.07540799999999985
-0.05273159220814705 0.07559859999999978
-0.05273159220814705 0.07552919999999978
-0.05273159220814705 0.07545979999999977
-0.027375560253858566 0.07578819999999993
-0.027375560253858566 0.07572659999999994
-0.027375560253858566 0.07566499999999994
0.013735469430685043 0.0758583999999999
0.013735469430685043 0.07580179999999989
0.013735469430685043 0.07574519999999989
-0.05455475300550461 0.07587219999999982
-0.05455475300550461 0.07581919999999982
-0.05455475300550461 0.07576619999999983
0.010069334879517555 0.07594779999999982
0.010069334879517555 0.07589939999999981
0.010069334879517555 0.07585099999999981
0.007141761481761

...

-0.28595733642578125 0.07749380000000006
-0.28595733642578125 0.07750280000000005
-0.28595733642578125 0.07751180000000006
-0.2168988138437271 0.0771536
-0.2168988138437271 0.07715540000000001
-0.2168988138437271 0.07715720000000001
-0.18990837037563324 0.07819940000000013
-0.18990837037563324 0.07822160000000013
-0.18990837037563324 0.07824380000000013
-0.2510707378387451 0.07758260000000008
-0.2510707378387451 0.07759140000000009
-0.2510707378387451 0.0776002000000001
-0.1610596626996994 0.0769868000000001
-0.1610596626996994 0.0769834000000001
-0.1610596626996994 0.0769800000000001
-0.16549555957317352 0.07739480000000004
-0.16549555957317352 0.07739960000000004
-0.16549555957317352 0.07740440000000005
-0.2420668601989746 0.07681760000000015
-0.2420668601989746 0.07681080000000015
-0.2420668601989746 0.07680400000000015
-0.28351259231567383 0.07658300000000011
-0.28351259231567383 0.07657200000000013
-0.28351259231567383 0.07656100000000013
-0.17927487194538116 0.07738640000000016
-

-0.128667950630188 0.07760960000000007
-0.128667950630188 0.07762780000000008
-0.19919319450855255 0.07742160000000006
-0.19919319450855255 0.07743540000000007
-0.19919319450855255 0.07744920000000008
-0.1576949805021286 0.07740180000000002
-0.1576949805021286 0.07741440000000001
-0.1576949805021286 0.07742700000000001
-0.15931041538715363 0.07792920000000014
-0.15931041538715363 0.07795140000000013
-0.15931041538715363 0.07797360000000013
-0.15504218637943268 0.07755720000000006
-0.15504218637943268 0.07757080000000005
-0.15504218637943268 0.07758440000000005
-0.14535805583000183 0.07727160000000009
-0.14535805583000183 0.07727880000000009
-0.14535805583000183 0.07728600000000009
-0.06647983938455582 0.07698720000000006
-0.06647983938455582 0.07698840000000005
-0.06647983938455582 0.07698960000000006
-0.07325580716133118 0.07676640000000005
-0.07325580716133118 0.07676320000000006
-0.07325580716133118 0.07676000000000006
-0.07572755217552185 0.07657320000000012
-0.07572755217552185 0.

-0.10974807292222977 0.07646759999999991
-0.10974807292222977 0.07645599999999991
-0.10974807292222977 0.07644439999999991
-0.12300476431846619 0.07635119999999991
-0.12300476431846619 0.07633799999999992
-0.12300476431846619 0.07632479999999991
-0.10003111511468887 0.07649520000000003
-0.10003111511468887 0.07648560000000001
-0.10003111511468887 0.07647600000000002
-0.016685282811522484 0.07655819999999998
-0.016685282811522484 0.07655039999999998
-0.016685282811522484 0.07654259999999997
-0.0954757034778595 0.076443
-0.0954757034778595 0.07643339999999998
-0.0954757034778595 0.07642379999999999
-0.03203609958291054 0.07647539999999985
-0.03203609958291054 0.07646699999999985
-0.03203609958291054 0.07645859999999986
-0.038202669471502304 0.07681739999999992
-0.038202669471502304 0.07681619999999992
-0.038202669471502304 0.07681499999999993
-0.0657220259308815 0.0767832
-0.0657220259308815 0.07678139999999999
-0.0657220259308815 0.07677959999999999
-0.035963188856840134 0.0766860000000

-0.05192158371210098 0.0765868
-0.05192158371210098 0.0765844
-0.07263091206550598 0.07649020000000004
-0.07263091206550598 0.07648600000000004
-0.07263091206550598 0.07648180000000004
-0.01765967160463333 0.0764572000000001
-0.01765967160463333 0.0764526000000001
-0.01765967160463333 0.0764480000000001
-0.04966071620583534 0.07649440000000017
-0.04966071620583534 0.07649080000000016
-0.04966071620583534 0.07648720000000017


127.0.0.1 - - [05/Nov/2017 21:27:51] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 252.467913
